# Mobility data converter

## Import modules

In [1]:
import sys
sys.path.insert(1, '../src/utils/')
from my_utils import *

import pandas as pd
import numpy as np

## Introduction

The goal is to convert data into the following format:

In [2]:
df_example = pd.read_csv('../data/processed/example_5nodes.csv')
df_example.head(2)

,timestamp,node_id,latitude,longitude,neighbours,rsu
0,1423807260,101,40.641606,-8.652702,[{'node_id': '102'| 'rssi': '46'| 'rsu': 0}],1
1,1423807260,102,40.641211,-8.653796,[{'node_id': '101'| 'rssi': '46'| 'rsu': 1}|{'...,0


## Variables

In [3]:
# columns from the original dataframe
cols = ['time_index', 'entity_id', 'location', 'speed', 'heading', 'altitude', 'rssi', 'obu_id',
        'road', 'class', 'test', 'receiverid', 'receivertype', 'stationid', 'stationtype',
        'semimajorconf', 'semiminorconf', 'semimajororient', 'altitudeconf', 'headingconf',
        'speedconf', 'vehiclerole', 'drivedirection', 'length', 'width', 'acceleration',
        'accelerationconf', 'curvature', 'curvatureconf', 'yawrate', 'yawrateconf', 'brakepedal',
        'gaspedal', 'emergencybrake', 'collisionwarning', 'accengaged', 'cruisecontrol',
        'speedlimiter', 'specialvehicle']

# selected features
features = ['time_index', 'entity_id', 'location', 'rssi', 'stationid']

In [4]:
# selected id's
rsu_ids = ['p3', 'p5', 'p6', 'p26', 'p19']
obu_ids = [50, 51, 52, 60, 86, 87, 89, 90, 97, 99]

In [5]:
rsu_locations = {'p3':(40.64074, -8.65705), 'p5':(40.64088, -8.65397), 'p6':(40.64161, -8.652827), 'p26':(40.63848, -8.65147), 'p19':(40.64339, -8.65847)}
rsu_locations

{'p3': (40.64074, -8.65705),
 'p5': (40.64088, -8.65397),
 'p6': (40.64161, -8.652827),
 'p26': (40.63848, -8.65147),
 'p19': (40.64339, -8.65847)}

In [6]:
# converts short id's to long id's
rsu_ids = build_posts_ids(rsu_ids)
rsu_ids

['urn:ngsi-ld:Values:aveiro_cam:p3',
 'urn:ngsi-ld:Values:aveiro_cam:p5',
 'urn:ngsi-ld:Values:aveiro_cam:p6',
 'urn:ngsi-ld:Values:aveiro_cam:p26',
 'urn:ngsi-ld:Values:aveiro_cam:p19']

In [7]:
# selected time period
data_init = '2022-03-07 08:00:00.00+00'
data_end = '2022-03-07 10:00:00.00+00'

In [8]:
# file containing the raw data
filename = '../data/raw/cams.part.01.csv'
# file for the processed data
filename_processed = '../data/processed/data.csv'

In [9]:
# radius (meters) for two entities being considered neighbours
radius = 50
# time for resample
time = '5s'

## Loading data

In [10]:
df = pd.read_csv(filename, names=cols) # read file

In [11]:
df['rssi'].describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.98])

count    1000000.000000
mean      -17322.096087
std        28832.587239
min       -65535.000000
10%       -65535.000000
20%       -65535.000000
30%          -88.000000
40%          -86.000000
50%          -84.000000
60%          -82.000000
70%          -79.000000
80%          -74.000000
90%          -67.000000
98%          -63.000000
max            0.000000
Name: rssi, dtype: float64

## Converting data

In [12]:
df = df[features] # select columns
df = df.loc[df['entity_id'].isin(rsu_ids)] # select only relevant rsu's
df = df.loc[df['stationid'].isin(obu_ids)] # select only relevant obu's
df = df.loc[(df['time_index']>=data_init) & (df['time_index']<data_end)] # include data_init and exclude data_end
df = df.loc[(df['rssi']!=0)] # filter by rssi values
df = df.sort_values(by='time_index')
df['time_index'] = pd.to_datetime(df['time_index'])
df["time_index"] = df["time_index"].dt.round(time) # round time_index columns
df = df.reset_index(drop=True) # reset index

df.head(2)

,time_index,entity_id,location,rssi,stationid
0,2022-03-07 08:01:00+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,0101000000D0381E7DDD4B21C0270BA9EC05524440,-89,87
1,2022-03-07 08:01:00+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,01010000007F9763C3E44B21C04B0169FF03524440,-89,87


In [13]:
df = df.groupby(['time_index', 'entity_id', 'stationid'], as_index=False).agg({'location':'first', 'rssi':'mean'})
df

,time_index,entity_id,stationid,location,rssi
0,2022-03-07 08:01:00+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,87,0101000000D0381E7DDD4B21C0270BA9EC05524440,-89.000000
1,2022-03-07 08:01:05+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,87,010100000099507754EB4B21C052DE7D4402524440,-87.833333
2,2022-03-07 08:01:10+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,87,01010000005AB09EA40A4C21C0153B1A87FA514440,-86.600000
3,2022-03-07 08:01:15+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,87,0101000000279133034B4C21C0FD6BD49EED514440,-86.666667
4,2022-03-07 08:01:20+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,87,0101000000277854466D4C21C08462D0AEE7514440,-86.800000
...,...,...,...,...,...
658,2022-03-07 09:52:10+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,86,01010000002943B005184D21C007A0072DDA514440,-85.400000
659,2022-03-07 09:52:15+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,86,0101000000CE7234A20F4D21C0632BC313D5514440,-84.000000
660,2022-03-07 09:52:20+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,86,0101000000027DC7951F4D21C096F551FCCE514440,-76.400000
661,2022-03-07 09:52:25+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,86,01010000006626625B504D21C000DF1225C6514440,-78.800000


In [14]:
df['gps'] = df['location'].map(convert_wkb_to_lat_lon) # convert location to gps
for n, col in enumerate(['latitude', 'longitude']): # extract lat, lon from gps
    df[col] = df['gps'].apply(lambda location: location[n])
df.head(2)

,time_index,entity_id,stationid,location,rssi,gps,latitude,longitude
0,2022-03-07 08:01:00+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,87,0101000000D0381E7DDD4B21C0270BA9EC05524440,-89.000000,"(40.6408058, -8.6481742)",40.640806,-8.648174
1,2022-03-07 08:01:05+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,87,010100000099507754EB4B21C052DE7D4402524440,-87.833333,"(40.6406942, -8.6482798)",40.640694,-8.648280


In [15]:
df = df.drop(columns=['location'])
df.head(2)

,time_index,entity_id,stationid,rssi,gps,latitude,longitude
0,2022-03-07 08:01:00+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,87,-89.000000,"(40.6408058, -8.6481742)",40.640806,-8.648174
1,2022-03-07 08:01:05+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,87,-87.833333,"(40.6406942, -8.6482798)",40.640694,-8.648280


In [16]:
# add rsu location
df['gps_rsu'] = df['entity_id'].str.split(':', expand=True)[4].apply(lambda pid: rsu_locations.get(pid))
for n, col in enumerate(['latitude_rsu', 'longitude_rsu']): # extract lat, lon from gps
    df[col] = df['gps_rsu'].apply(lambda location: location[n])
df

,time_index,entity_id,stationid,rssi,gps,latitude,longitude,gps_rsu,latitude_rsu,longitude_rsu
0,2022-03-07 08:01:00+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,87,-89.000000,"(40.6408058, -8.6481742)",40.640806,-8.648174,"(40.63848, -8.65147)",40.63848,-8.65147
1,2022-03-07 08:01:05+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,87,-87.833333,"(40.6406942, -8.6482798)",40.640694,-8.648280,"(40.63848, -8.65147)",40.63848,-8.65147
2,2022-03-07 08:01:10+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,87,-86.600000,"(40.640458, -8.6485187)",40.640458,-8.648519,"(40.63848, -8.65147)",40.63848,-8.65147
3,2022-03-07 08:01:15+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,87,-86.666667,"(40.6400641, -8.6490098)",40.640064,-8.649010,"(40.63848, -8.65147)",40.63848,-8.65147
4,2022-03-07 08:01:20+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,87,-86.800000,"(40.6398829, -8.6492712)",40.639883,-8.649271,"(40.63848, -8.65147)",40.63848,-8.65147
...,...,...,...,...,...,...,...,...,...,...
658,2022-03-07 09:52:10+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,86,-85.400000,"(40.6394707, -8.6505739)",40.639471,-8.650574,"(40.63848, -8.65147)",40.63848,-8.65147
659,2022-03-07 09:52:15+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,86,-84.000000,"(40.6393151, -8.6505099)",40.639315,-8.650510,"(40.63848, -8.65147)",40.63848,-8.65147
660,2022-03-07 09:52:20+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,86,-76.400000,"(40.6391292, -8.6506316)",40.639129,-8.650632,"(40.63848, -8.65147)",40.63848,-8.65147
661,2022-03-07 09:52:25+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,86,-78.800000,"(40.6388594, -8.6510037)",40.638859,-8.651004,"(40.63848, -8.65147)",40.63848,-8.65147


In [17]:
def rsu_to_node_id(df):
    _df = df.copy()
    _df = _df[['time_index', 'entity_id', 'latitude_rsu', 'longitude_rsu']]
    _df = _df.rename(columns={'entity_id':'node_id', 'latitude_rsu': 'latitude', 'longitude_rsu': 'longitude'})
    _df['neighbours'] = df.apply(lambda x : [(x.stationid, x.rssi, 0)], axis=1)
    _df['rsu'] = [1]*len(_df.index)
    _df = _df.groupby(['time_index', 'node_id', 'latitude', 'longitude'], as_index=False).agg({'neighbours': 'sum', 'rsu': 'first'})
    # 'neighbours': 'sum' -> sum of lists
    return _df

In [18]:
df_rsus = rsu_to_node_id(df)
df_rsus.head()

,time_index,node_id,latitude,longitude,neighbours,rsu
0,2022-03-07 08:01:00+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,40.63848,-8.65147,"[(87, -89.0, 0)]",1
1,2022-03-07 08:01:05+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,40.63848,-8.65147,"[(87, -87.83333333333333, 0)]",1
2,2022-03-07 08:01:10+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,40.63848,-8.65147,"[(87, -86.6, 0)]",1
3,2022-03-07 08:01:15+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,40.63848,-8.65147,"[(87, -86.66666666666667, 0)]",1
4,2022-03-07 08:01:20+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,40.63848,-8.65147,"[(87, -86.8, 0)]",1


In [19]:
def obu_to_node_id(df):
    _df = df.copy()
    _df = _df[['time_index', 'stationid', 'latitude', 'longitude']]
    _df = _df.rename(columns={'stationid':'node_id'})
    _df['neighbours'] = df.apply(lambda x : [(x.entity_id, x.rssi, 1)], axis=1)
    _df['rsu'] = [0]*len(_df.index)
    _df = _df.groupby(['time_index', 'node_id', 'latitude', 'longitude'], as_index=False).agg({'neighbours': 'sum', 'rsu': 'first'})
    # 'neighbours': 'sum' -> sum of lists
    return _df

In [20]:
df_obus = obu_to_node_id(df)
df_obus.head()

,time_index,node_id,latitude,longitude,neighbours,rsu
0,2022-03-07 08:01:00+00:00,87,40.640806,-8.648174,"[(urn:ngsi-ld:Values:aveiro_cam:p26, -89.0, 1)]",0
1,2022-03-07 08:01:05+00:00,87,40.640694,-8.648280,"[(urn:ngsi-ld:Values:aveiro_cam:p26, -87.83333...",0
2,2022-03-07 08:01:10+00:00,87,40.640458,-8.648519,"[(urn:ngsi-ld:Values:aveiro_cam:p26, -86.6, 1)]",0
3,2022-03-07 08:01:15+00:00,87,40.640064,-8.649010,"[(urn:ngsi-ld:Values:aveiro_cam:p26, -86.66666...",0
4,2022-03-07 08:01:20+00:00,87,40.639883,-8.649271,"[(urn:ngsi-ld:Values:aveiro_cam:p26, -86.8, 1)]",0


In [21]:
# Concatenate df_rsus with df_obus
df_nodes = pd.concat([df_rsus, df_obus])
df_nodes = df_nodes.sort_values(by='time_index')
df_nodes.head()

,time_index,node_id,latitude,longitude,neighbours,rsu
0,2022-03-07 08:01:00+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,40.638480,-8.651470,"[(87, -89.0, 0)]",1
0,2022-03-07 08:01:00+00:00,87,40.640806,-8.648174,"[(urn:ngsi-ld:Values:aveiro_cam:p26, -89.0, 1)]",0
1,2022-03-07 08:01:05+00:00,87,40.640694,-8.648280,"[(urn:ngsi-ld:Values:aveiro_cam:p26, -87.83333...",0
1,2022-03-07 08:01:05+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,40.638480,-8.651470,"[(87, -87.83333333333333, 0)]",1
2,2022-03-07 08:01:10+00:00,87,40.640458,-8.648519,"[(urn:ngsi-ld:Values:aveiro_cam:p26, -86.6, 1)]",0


In [22]:
def list_to_neighbour_format(l):
    txt = "["
    for i in range(len(l)):
        t = l[i]
        txt += "{'node_id': '" + str(t[0]) + "'| 'rssi': '" + str(abs(t[1])) + "'| 'rsu': " + str(t[2]) + "}"
        if i<len(l)-1:
            txt += ","
        
    txt += "]"
    return txt

In [23]:
df_nodes['neighbours'] = df_nodes['neighbours'].apply(lambda x :list_to_neighbour_format(x))
df_nodes.head()

,time_index,node_id,latitude,longitude,neighbours,rsu
0,2022-03-07 08:01:00+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,40.638480,-8.651470,[{'node_id': '87'| 'rssi': '89.0'| 'rsu': 0}],1
0,2022-03-07 08:01:00+00:00,87,40.640806,-8.648174,[{'node_id': 'urn:ngsi-ld:Values:aveiro_cam:p2...,0
1,2022-03-07 08:01:05+00:00,87,40.640694,-8.648280,[{'node_id': 'urn:ngsi-ld:Values:aveiro_cam:p2...,0
1,2022-03-07 08:01:05+00:00,urn:ngsi-ld:Values:aveiro_cam:p26,40.638480,-8.651470,[{'node_id': '87'| 'rssi': '87.83333333333333'...,1
2,2022-03-07 08:01:10+00:00,87,40.640458,-8.648519,[{'node_id': 'urn:ngsi-ld:Values:aveiro_cam:p2...,0


## Saving data

In [24]:
df_nodes.to_csv(filename_processed)